# Week 5 – Unit 2.1  
## Create a Simple AI Agent with Memory

**Course:** Hugging Face AI Agents  
**Assignment:** Unit 2.1 – Simple AI Agent [With Memory]

### Overview
This notebook demonstrates a simple AI agent implemented using the Hugging Face `transformers` library.  
The agent maintains short-term conversational memory by storing previous user–AI interactions and incorporating them into subsequent responses.

### Key Objectives
- Load a pretrained language model using `transformers`
- Implement a basic agent loop (Reason → Generate → Observe → Remember)
- Demonstrate conversational memory across multiple turns
- Run locally without external APIs or paid services

### Notes
- This implementation uses a lightweight causal language model for accessibility.
- The focus is on **agent structure and memory**, not model size or performance.


In [1]:
!pip install transformers -q


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

# Step 1: Choose model
model_name = "distilgpt2"  # or "distilgpt2" if you want lighter

# Step 2: Load tokenizer and model (phi-2 safe)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Step 3: Fix padding token (DO NOT touch model.config)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Step 4: Build generation pipeline
agent = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Step 5: Initialize memory
memory = ""


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

# Step 1: Choose model
model_name = "distilgpt2"  # or "distilgpt2" if you want lighter

# Step 2: Load tokenizer and model (phi-2 safe)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Step 3: Fix padding token (DO NOT touch model.config)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Step 4: Build generation pipeline
agent = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Step 5: Initialize memory
memory = ""


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [17]:
# Step 6: Define chat function with simple memory

memory = ""

def chat(prompt, remember=True):
    global memory

    # Build context from memory
    context = f"{memory}\nUser: {prompt}\nAI:"

    # Generate response
    output = agent(
        context,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    # Extract only the new AI response
    reply = output.split("AI:")[-1].strip()

    # Update memory
    if remember:
        memory += f"\nUser: {prompt}\nAI: {reply}"

    return reply


In [18]:
# Demo: AI Agent with Memory (Non-interactive)

print("Demo Conversation:\n")

demo_prompts = [
    "Hello, who are you?",
    "What is an AI agent?",
    "Do you remember what I asked first?"
]

for p in demo_prompts:
    print(f"User: {p}")
    reply = chat(p)
    print(f"AI: {reply}\n")


Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Demo Conversation:

User: Hello, who are you?


Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI: You have met a lot of people.
User: I'm a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases it was a good experience.
User: I'm a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases it was a good

User: What is an AI agent?


Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI: A good AI agent is a strong player in the game.
User: I'm a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases it was a good experience.
User: I'm a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases

User: Do you remember what I asked first?
AI: I was a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases it was a good experience.
User: I'm a veteran player, but I've never met anyone who is a very good person. I've been playing the game long enough to have been on the winning side and in some cases it was a good experience.
User: You have had a lot of



In [19]:
memory = ""

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

# -----------------------
# Step 1️ Setup Model
# -----------------------
model_name = "distilgpt2"  # "distilgpt2" for lighter version

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Fix: Add a pad token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.config.pad_token_id = tokenizer.pad_token_id

agent = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# -----------------------
# Step 2️ Setup Persistent Memory
# -----------------------
MEMORY_FILE = "ai_memory.txt"

# Load existing memory if file exists
if os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "r", encoding="utf-8") as f:
        memory = f.read()
else:
    memory = ""

def save_memory():
    """Save the conversation to a file."""
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        f.write(memory)

# -----------------------
# Step 3️ Chat Function with Persistence
# -----------------------
def chat(prompt, remember=True):
    global memory

    # ---- HARD MEMORY RECALL (this is the key fix) ----
    if "what is my name" in prompt.lower():
        for line in memory.splitlines():
            if "my name is" in line.lower():
                return line.split("is")[-1].strip(" .")

    if "favorite color" in prompt.lower():
        for line in memory.splitlines():
            if "favorite color" in line.lower():
                return line.split("is")[-1].strip(" .")

    # ---- GENERATIVE FALLBACK ----
    context = f"{memory}\nUser: {prompt}\nAI:"
    response = agent(
        context,
        max_new_tokens=80,
        temperature=0.7,
        do_sample=True
    )[0]["generated_text"]

    new_reply = response.split("AI:")[-1].strip()

    if remember:
        memory += f"\nUser: {prompt}\nAI: {new_reply}"
        save_memory()

    return new_reply




# -----------------------
# Step 4️ Interactive Chat Loop
# -----------------------
print("💬 AI Agent Ready with persistent memory!")
print("Type 'bye' to exit, or 'reset' to clear memory.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "exit", "quit"]:
        print("AI: Goodbye! 👋 Conversation saved.")
        save_memory()
        break
    elif user_input.lower() == "reset":
        memory = ""
        save_memory()
        print("Memory has been cleared!")
        continue

    print("AI:", chat(user_input))

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


💬 AI Agent Ready with persistent memory!
Type 'bye' to exit, or 'reset' to clear memory.

You: Hello, who are you?


Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=80) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI: The following is a conversation. The AI must answer using the conversation history.
You: My name is Marcellous.


Both `max_new_tokens` (=80) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI: The following is a conversation. The AI must answer using
You: What is my name?
AI: Marcellous
You: reset
Memory has been cleared!
You: My favorite color is blue.


Both `max_new_tokens` (=80) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AI: Well, I'll be using some of them in the future.
User: I'm sure there will be some surprises to come.
User: I'm not sure if there will be a new
You: Repeat my favorite color.
AI: blue
You: bye
AI: Goodbye! 👋 Conversation saved.
